In [1]:
import time
import argparse
import os
import pickle

import gym
from stable_baselines3 import PPO

import l5kit.environment
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset, filter_agents_by_frames
from l5kit.dataset import EgoDataset
from l5kit.environment.envs.l5_env import SimulationConfigGym
from l5kit.environment.gym_metric_set import L2DisplacementYawMetricSet
from l5kit.environment.policy_utils import rollout_multiple_scenes
from l5kit.rasterization import build_rasterizer
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from l5kit.data import MapAPI

from bokeh.io import output_notebook, show
from prettytable import PrettyTable


In [2]:
os.environ["L5KIT_DATA_FOLDER"] = "/home/ubuntu/level5_data/"
# set env variable for data
dm = LocalDataManager(None)
# get config
cfg = load_config_data("../../../examples/RL/config.yaml")

mapAPI = MapAPI.from_cfg(dm, cfg)

env_config_path = '../../../examples/RL/config.yaml'

# make env of 32 time-steps and 64 time-steps
env = gym.make("L5-CLE-v0", env_config_path=env_config_path, use_kinematic=True, return_info=True, train=False)

# sim_cfg = SimulationConfigGym(False, True, False, 30.0, 15.0, 0, 65)
# print("NamedTuple: ", sim_cfg)
# print("STEP: ", sim_cfg.num_simulation_steps)
# env_64 = gym.make("L5-CLE-v0", env_config_path=env_config_path, sim_cfg=sim_cfg, use_kinematic=True, return_info=True)

output_notebook()

/home/ubuntu/src/l5kit/l5kit/l5kit/data/zarr_dataset.py:226: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  dataset = ChunkedDataset("")
/home/ubuntu/src/l5kit/l5kit/l5kit/simulation/utils.py:107: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  new_dataset = ChunkedDataset("")


Loading BokehJS ...

In [3]:
def get_simulation_outputs(model_path, env, start=0, end=1):
    t_step = model_path.split('_')[-2]
    print("Time Step:", t_step)
    model = PPO.load(model_path, env)
    print("seed: ", model.seed)
    sim_outs = rollout_multiple_scenes(model, env, list(range(start, end)))
    return sim_outs

def visualize_outputs(sim_outs):
    for sim_out in sim_outs: # for each scene
        print("Scene ID: ", sim_out.scene_id)
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, mapAPI)
        show(visualize(sim_out.scene_id, vis_in))


def quantify_outputs(sim_outs, metric_set=None):
    metric_set = metric_set if metric_set is not None else L2DisplacementYawMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE", "ADE"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    tot_ade = 0.0
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        ade_error = scene_metrics["displacement_error_l2"][1:].mean()
        fde_error = scene_metrics['displacement_error_l2'][-1]
        table.add_row([scene_id, round(fde_error.item(), 4), round(ade_error.item(), 4)])
        tot_fde += fde_error.item()
        tot_ade += ade_error.item()

    ave_fde = tot_fde / len(scene_results)
    ave_ade = tot_ade / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4), round(ave_ade, 4)])
    print(table)

from moviepy.editor import ImageSequenceClip
from IPython.display import display, Image

def gif_outputs(sim_outs, name='test'):
    sim_out = sim_outs[0] # for each scene
    print("Scene ID: ", sim_out.scene_id)
    vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, mapAPI)
    gif_frames = visualize_gif(sim_out.scene_id, vis_in)

    # save it as a gif
    clip = ImageSequenceClip(list(gif_frames), fps=20)
    clip.write_gif(name + '.gif', fps=20)

In [7]:
from l5kit.environment.gym_metric_set import CLEMetricSet
from l5kit.cle.validators import ValidationCountingAggregator

def evaluate_scenes(model_path, env, start=0, end=1) -> None:
    """Evaluate the episode outputs for `n_eval_episodes` episodes.
    """
    t_step = model_path.split('_')[-2]
    print("Time Step:", t_step)
    model = PPO.load(model_path, env)
    print("seed: ", model.seed)
    metric_set = CLEMetricSet()

    def loop_over(model, env, start, end):
        idx = start
        env.reset_scene_id = idx
        obs = env.reset()
        while True:
            action, _ = model.predict(obs, deterministic=True)
            obs, _, done, info = env.step(action)

            if done:
                metric_set.evaluate(info["sim_outs"])

                # New Episode
                idx += 1
                env.reset_scene_id = idx
                obs = env.reset()
                
                # End
                if idx == end:
                    return
    
    # loop
    loop_over(model, env, start, end)

    # Aggregate
    validation_results = metric_set.evaluator.validation_results()
    agg = ValidationCountingAggregator().aggregate(validation_results)

    for k, v in agg.items():
        print(f'eval/{k}: ', v.item())

        

            # Vectorize
#             for idx in range(self.n_eval_envs):
#                 if done[idx]:
#                     episodes_done += 1
#                     metric_set.evaluate(info[idx]["sim_outs"])

#                     if episodes_done == self.n_eval_episodes:
#                         return

### Overfitting 1 scene of 32 time-steps (Group Normalization + Kinematic Model)

In [ ]:
# files = ['./logs/verify_eps32_simple_gn_km_re_400000_steps.pkl']
model_path = './logs/verify_eps32_simple_gn_km_re_400000_steps'
sim_outs = get_simulation_outputs(model_path, env)
# visualize_outputs(files)
quantify_outputs(sim_outs)

### Overfitting 5 scenes of 32 time-steps (Group Normalization + Kinematic Model)

In [ ]:
files = ['./logs/5scenes_kinematic_1M_history3_gn2_950000_steps.pkl']
# visualize_outputs(files)
quantify_outputs(files)

### Overfitting 10 scenes of 32 time-steps (Group Normalization + Kinematic Model)

In [ ]:
files = ['./logs/10scenes_kinematic_1M_history3_gn2_950000_steps.pkl']
# visualize_outputs(files)
quantify_outputs_viz(files)

In [ ]:
model_path = './logs/10scenes_kinematic_1M_history3_gn2_950000_steps'
sim_outs = get_simulation_outputs(model_path, env, 0, 10)
# visualize_outputs(files)
quantify_outputs(sim_outs)

### Overfitting 1 scenes of 32 time-steps at a TURN (Kinematic Model)

In [ ]:
files = ['./logs/turn_f27_hyp_g95_rollout_steps256_rescale_auto_yawwt10_1M_200000_steps.pkl']
# visualize_outputs(files)
quantify_outputs(files)

### Overfitting 1 scenes of 64 time-steps (Group Normalization + Kinematic Model)

In [ ]:
# History 1, Accn: 0.3, Angle: 15 degrees
name = 'straight_64'
start = 0
end = 1
model_files = [
               './logs/verify_eps64_simple_gn2_km_500000_steps',
              ]

for model_file in model_files:
    sim_outs = get_simulation_outputs(model_file, env_64, start=start, end=end)
    quantify_outputs(sim_outs)
    visualize_outputs(sim_outs)
    gif_outputs(sim_outs, name)


### Curriculum Learning: Overfitting 1 scenes of 64 time-steps (Group Normalization + Kinematic Model)

In [ ]:
files = ['./logs/verify_eps64_simple_gn2_km_curriculum_400000_steps.pkl']
# visualize_outputs(files)
quantify_outputs(files)

### Overfitting 1 scenes of 64 time-steps at a TURN (Kinematic Model)

In [ ]:
# files = ['./logs/verify_eps64_simple_gn2_km_turn_1000000_steps.pkl']
# visualize_outputs(files)
# quantify_outputs(files)

# History 3, Accn: 0.3, Angle: 30 degrees
name = 'turn_64'
start = 8
end = 9
model_files = [
               './logs/verify_eps64_simple_gn2_km_turn_1000000_steps',
              ]

for model_file in model_files:
    sim_outs = get_simulation_outputs(model_file, env_64, start=start, end=end)
    quantify_outputs(sim_outs)
    visualize_outputs(sim_outs)
    gif_outputs(sim_outs, name)


In [ ]:
len(env_64.sim_dataset.scene_dataset_batch[0].dataset.frames)



### Training Evolution: Overfitting 1 scenes of 64 time-steps at a TURN (Kinematic Model)

In [ ]:
files = ['./logs/verify_eps64_simple_gn2_km_turn_50000_steps.pkl',
         './logs/verify_eps64_simple_gn2_km_turn_100000_steps.pkl',
         './logs/verify_eps64_simple_gn2_km_turn_150000_steps.pkl',
         './logs/verify_eps64_simple_gn2_km_turn_200000_steps.pkl',
         './logs/verify_eps64_simple_gn2_km_turn_250000_steps.pkl',
         './logs/verify_eps64_simple_gn2_km_turn_300000_steps.pkl',
         './logs/verify_eps64_simple_gn2_km_turn_1000000_steps.pkl',
        ]
# visualize_outputs(files)
quantify_outputs(files)

### Overfitting 42 scenes of 32 time-steps (Group Normalization + Kinematic Model)

In [ ]:
files = ['./logs/42scenes_kinematic_1M_3M_history3_gn2_c01_1000000_steps.pkl',
         './logs/42scenes_kinematic_1M_3M_history3_gn2_c01_1800000_steps.pkl',
         './logs/42scenes_kinematic_1M_history3_gn2_c01_gae090_1000000_steps.pkl',
         './logs/42scenes_kinematic_1M_history3_gn2_c01_gamma099_1000000_steps.pkl',
         './logs/42scenes_kinematic_2M_history3_gn2_c02_2000000_steps.pkl']
# visualize_outputs(files)
quantify_outputs(files)

In [ ]:


# model_file = './logs/42scenes_kinematic_1M_3M_history3_gn2_c01_1800000_steps'
model_file = './logs/42scenes_kinematic_1M_history3_gn2_c01_gae090_1000000_steps'

model = PPO.load(model_file, env)



In [ ]:
sim_outs = rollout_multiple_scenes(model, env, list(range(42)))
get_metrics_table(sim_outs, None)

In [ ]:
model_file = './logs/42scenes_kinematic_1M_history3_gn2_c01_gae090_1000000_steps'
sim_outs = get_simulation_outputs(model_file, env, start=0, end=42)
quantify_outputs(sim_outs)

In [ ]:
# def get_simulation_outputs(model_path, env, start=0, end=1):
#     t_step = model_path.split('_')[-2]
#     print("Time Step:", t_step)
#     model = PPO.load(model_path, env)
#     sim_outs = rollout_multiple_scenes(model, env, list(range(start, end)))
#     return sim_outs

def visualize_outputs(files):
    for file in files:
        t_step = file.split('_')[-2]
        print("Time Step:", t_step)
        with open(file, 'rb') as f:
            sim_outs = pickle.load(f)
    for sim_out in sim_outs: # for each scene
        print("Scene ID: ", sim_out.scene_id)
        vis_in = (sim_out, mapAPI)
        show(visualize(sim_out.scene_id, vis_in))


def get_metrics_table(sim_outs, metric_set):
    metric_set = metric_set if metric_set is not None else L5GymCLEMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        dist_error = scene_metrics['displacement_error_l2'][-1]
        table.add_row([scene_id, round(dist_error.item(), 4)])
        tot_fde += dist_error.item()
    ave_fde = tot_fde / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4)])
    print(table)

def quantify_outputs_viz(files, metric_set=None):
    for file in files:
        t_step = file.split('_')[-2]
        print("Time Step:", t_step)
        with open(file, 'rb') as f:
            sim_outs = pickle.load(f)
            get_metrics_table(sim_outs, metric_set)

### Overfitting 10 scenes of 32 time-steps (Group Normalization + Kinematic Model)

In [ ]:
## Steer, acc = math.radians(25) * 0.1, 0.3
start = 1000
end = 1100
model_files = [
               './logs/10scenes_kinematic_1M_history3_gn2_1000000_steps',
              ]
for model_file in model_files:
    sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
    quantify_outputs(sim_outs)
#     visualize_outputs(sim_outs)


### Overfitting 42 scenes of 32 time-steps (Group Normalization + Kinematic Model)

#### Long Training of 8M time steps: 36 hours

In [ ]:
name = 'red_light'
start = 3
end = 10
model_files = [
               './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_10000000_steps',
#                './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_9000000_steps',
#                 './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_8000000_steps',
#                './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_7000000_steps',
#                './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_6000000_steps',
#                './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_5000000_steps',
#                './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_4000000_steps',
#                './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_3000000_steps',
#                './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_2000000_steps',
#                './logs/42scenes_kinematic_10M_history3_gn2_c01_c0001_gae090_1000000_steps'
              ]

for model_file in model_files:
    sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
    quantify_outputs(sim_outs)
#     sim_outs = sim_outs[0:10]
#     visualize_outputs(sim_outs)
    gif_outputs(sim_outs, name)


In [ ]:
start = 1000
end = 1100
model_files = [
#                './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_16000000_steps',
#                './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_14000000_steps',
#                './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_12000000_steps',
#                './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_12000000_steps',
#                './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_11000000_steps',
#                './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_10000000_steps',
#                './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_9000000_steps',
#                './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_8000000_steps',
              ]

for model_file in model_files:
    sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
    quantify_outputs(sim_outs)
#     sim_outs = sim_outs[11:12]
#     visualize_outputs(sim_outs)

In [ ]:
name = 'const_2'
start = 1097
end = 1098
model_files = [
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_18000000_steps',
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_16000000_steps',
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_14000000_steps',
               './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_13000000_steps',
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_12000000_steps',
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_11000000_steps',
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_10000000_steps',
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_9000000_steps',
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_8000000_steps',
              ]

for model_file in model_files:
    sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
    quantify_outputs(sim_outs)
#     sim_outs = sim_outs[11:12]
    visualize_outputs(sim_outs)
    gif_outputs(sim_outs, name)
 

## Validation Set Quantify

In [6]:
start = 0
end = 100
model_files = [
               './logs/100scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_16000000_steps',
               './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_18000000_steps',
#                './logs/16000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_20000000_steps',                
              ]

for model_file in model_files:
    sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
    quantify_outputs(sim_outs)

Time Step: 16000000
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
seed:  42
+----------+---------+---------+
| scene_id |   FDE   |   ADE   |
+----------+---------+---------+
|    0     |  0.1139 |  0.1091 |
|    1     |  2.4261 |  1.3972 |
|    2     |  4.7675 |  2.1369 |
|    3     |  4.7727 |  3.1344 |
|    4     |  5.0418 |  2.7709 |
|    5     |  4.672  |  1.816  |
|    6     |  1.9848 |  0.8553 |
|    7     |  4.4706 |  1.938  |
|    8     | 11.6853 |  3.3599 |
|    9     |  7.1361 |  5.4318 |
|    10    |  2.5421 |  1.4162 |
|    11    |  3.1605 |  1.5119 |
|    12    |  4.1031 |  2.9686 |
|    13    |  0.6652 |  0.5068 |
|    14    |  4.4183 |  0.9754 |
|    15    |  2.3556 |  0.5658 |
|    16    | 19.8549 |  8.2567 |
|    17    |  0.8711 |  0.3929 |
|    18    |  3.545  |  1.4154 |
|    19    |  5.2995 |  2.8567 |
|    20    |  0.632  |  0.3389 |
|    21    |  2.4871 |  1.3234 |
|    22    | 33.1938 | 14.7904 |
|    23    |  1.8182 |  1.0934 |
| 

In [12]:
start = 0
end = 100
model_files = [
#                './logs/1000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_18000000_steps',
               './logs/16000scenes_kinematic_20M_history3_gn2_c01_c0001_10M_gae090_20000000_steps',                
              ]

for model_file in model_files:
    sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
    quantify_outputs(sim_outs)
    evaluate_scenes(model_file, env, start=start, end=end)


Time Step: 20000000
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
seed:  42
+----------+--------+--------+
| scene_id |  FDE   |  ADE   |
+----------+--------+--------+
|    0     | 0.4562 | 0.1554 |
|    1     | 1.0638 | 0.3843 |
|    2     | 1.1445 | 0.3995 |
|    3     | 0.8278 | 0.3176 |
|    4     | 0.9897 | 0.5629 |
|    5     | 1.4305 | 0.4906 |
|    6     | 1.3871 | 0.4751 |
|    7     | 1.4204 | 0.4882 |
|    8     | 1.3853 | 0.4771 |
|    9     | 2.9048 | 1.0831 |
|    10    | 0.8712 | 0.3216 |
|    11    | 4.5204 | 1.5446 |
|    12    | 1.5092 | 0.5612 |
|    13    | 4.9848 | 1.8635 |
|    14    | 8.9953 | 3.1702 |
|    15    | 0.8695 |  0.45  |
|    16    | 1.7968 | 0.4861 |
|    17    | 1.1087 | 0.3807 |
|    18    | 1.1102 | 0.3816 |
|    19    | 2.2272 | 0.9431 |
|    20    | 1.4689 | 0.5984 |
|    21    | 1.2721 | 0.4762 |
|    22    | 1.8605 | 0.6472 |
|    23    | 1.0912 | 0.426  |
|    24    | 6.7768 | 2.2524 |
|    25    | 3.1952 | 1.1

In [ ]:
2.22, 0.78

In [ ]:
from IPython.display import display, Image
Image(name + '.gif')